## EDA for Hours Spent on Labs Survey

In [3]:
suppressPackageStartupMessages(library(tidyverse))

This analysis will investigate the following question:

*How does the number of times a Master of Data Science (MDS) student goes to office hours affect the average number of hours they spend working on labs per week?*

based on the results collected from this [survey](https://ubc-mds.slack.com/archives/C24HU8X0W/p1554330074049900). We stored our raw data in this [repo]().

In [4]:
# Load survey data
rawdata = read.csv("Hours_Spent_On_Labs_Survey_Final.csv") 

In [19]:
# Data cleaning
raw_df = data.frame(rawdata)
row_to_remove = c(1:2)
col_to_remove = c(1:19, 21, 25, 30, 31)
df = raw_df[-row_to_remove, -col_to_remove]
names(df) <- c("attend_OH", "lab_hours", "group", "academic", "yrs_out_school", "program", "stat", "optional")
df <- df %>% mutate(group = as.factor(group), academic = as.factor(academic), yrs_out_school= as.integer(yrs_out_school)) %>% select (lab_hours, everything())


In [20]:
df

lab_hours,attend_OH,group,academic,yrs_out_school,program,stat,optional
20,0,Mostly alone,Engineering,2,3,2,3
20,2,Mostly alone,Science,2,3,3,4
15,0,Mostly alone,Business,9,4,2,1
19,4,Mostly in groups,Math,13,3,3,1
20,0,Mostly alone,Engineering,2,4,3,1
30,5,Mostly in groups,Math,13,2,2,3
20,0,Mostly in groups,Math,2,3,4,3
15,0,Mostly in groups,Math,12,2,5,2
50,3,Mostly in groups,Math,10,2,4,2
36,2,Mostly in groups,Engineering,5,1,3,2


In [46]:
# write.csv(df, file = "clean_data.csv")